In [1]:
#!/usr/bin/env python
# coding: utf-8

# # Embedding Network

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd

In [2]:
class EmbeddingModel(nn.Module):
    def __init__(self, datamap, y_col = "y"):
        super(EmbeddingModel, self).__init__()
        
        self.datamap = datamap
        self.y_col = y_col

        # 임베딩 레이어 초기화
        for k, v in datamap.items() :
            if v == "linear" :
                setattr(self, k, nn.Linear(num_linear_features, embedding_dim))
            elif v == "onehot" :
                setattr(self, k, nn.Embedding(num_sparse_features, embedding_dim))
            elif v == "multihot" :
                setattr(self, k, nn.Embedding(num_sparse_features, embedding_dim))
        fc1_embedding_dim = len(datamap) - 1
        # 다층 퍼셉트론(MLP) 레이어 초기화 
        self.fc1 = nn.Linear(embedding_dim * fc1_embedding_dim, hidden_dim)  # 임베딩된 특성이 3개이므로 *3
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def _get_multihot_embedding(self, embedding, method_multihot) :

        if method_multihot == "mean" :
            # 멀티-핫 특성의 임베딩 평균
            multihot_embedding = lambda x : torch.mean(embedding(x), dim = 1)
        elif method_multihot == "sum" :
            # 각 인덱스에 대한 임베딩 벡터를 더하여 합산
            multihot_embedding = lambda x : torch.sum(embedding(x), dim = 1)
            # multi_hot_embedded = torch.sum(embedding(x[k]), dim=1)
        elif method_multihot == "weighted_mean" :
            # 각 인덱스에 대한 임베딩 벡터를 가져오고 가중 평균 계산
            weights = torch.ones_like(emb_vectors)  # 간단히 모든 값에 대해 동일한 가중치 사용
            multihot_embedding = lambda x : torch.mean(embedding(x) * weights, dim=1)

        return multihot_embedding
        
    
    def forward(self, x, method_multihot = "sum"):
        # sparse feature 임베딩
        embedded = {}
        for k, v in self.datamap.items() :
            if k == self.y_col :
                continue

            embedding = getattr(self, k)

            if v == "multihot" :
                embedding = self._get_multihot_embedding(embedding, method_multihot)

            embedding_value = embedding(x[k])
            embedded[k] = embedding_value

        # 모든 임베딩된 특성을 결합
        combined_features = torch.cat([v for k, v in embedded.items()], dim=1)  # dim=1은 각 임베딩을 행 방향으로 결합
        
        # MLP 레이어 적용
        x = F.relu(self.fc1(combined_features))
        x = self.fc2(x)
        
        return x

In [15]:
class SiamesDataset(Dataset): 
    def __init__(self, df1, datamap1, df2, datamap2, df_y, y_col = "y"):
        
        self.x_data1 = self._get_x_data(df1, datamap1)
        self.x_data2 = self._get_x_data(df2, datamap2)
        self.y_data = torch.LongTensor(df_y[y_col].values)

  # 총 데이터의 개수를 리턴
    def __len__(self):
        return len(self.y_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx):
        x1 = {x : y[idx] for x, y in self.x_data1.items()}
        x2 = {x : y[idx] for x, y in self.x_data2.items()}
        y = self.y_data[idx]
        return x1, x2, y

    def _get_x_data(self, df, datamap) :
        
        x_data = {}
        for col, v in datamap.items() :
            if v == "linear" :
                x_data[col] = torch.FloatTensor(df[col].values).unsqueeze(1)
            if v == "multihot" :
                df = df.assign(**{col : lambda x : self._pad_sequence(x[col])})
                x_data[col] = torch.LongTensor(df[col].to_list())
            if v == "onehot" :
                x_data[col] = torch.LongTensor(df[col].values)

        return x_data
    
    def _get_max_multihot_size(self, series):
        return series.apply(len).max()

    def _get_max_multihot_value(self, series):
        return series.apply(max).max()
        
    def _pad_infinite(self, iterable, padding=None):
        from itertools import chain, repeat, islice
        return chain(iterable, repeat(padding))
    
    def _pad(self, iterable, size, padding=None):
        from itertools import chain, repeat, islice
        return list(islice(self._pad_infinite(iterable, padding), size))
        
    def _pad_sequence(self, series) :
        l = self._get_max_multihot_size(series)
        m = self._get_max_multihot_value(series)
        return series.apply(lambda x : self._pad(x, l, m + 1))

In [4]:
# # Siames Network
class SiameseNetwork(nn.Module):
    def __init__(self, model1, model2):
        super(SiameseNetwork, self).__init__()
        self.model1 = model1
        self.model2 = model2

    def forward(self, input1, input2):
        output1 = self.model1(input1)
        output2 = self.model2(input2)
        return output1, output2

In [5]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [6]:
import random

def make_onehot_data(length, *args) :
    return [random.randint(*args[0]) for _ in range(length)]

def make_linear_data(length, *args) :
    return [random.random() * args[0] for _ in range(length)]

def make_multihot(length, *args) :
    args = args[0]
    multihot_length = args[0]
    min_range = args[1]
    max_range = args[2]
    
    return [[random.randint(min_range, max_range) for _ in range(multihot_length)] for _ in range(length)]

def random_config(feature_type) :

    if feature_type == "onehot" :
        val1 = random.randint(1, 30)
        val2 = random.randint(1, 30)
        return min(val1, val2), max(val1, val2)
        
    if feature_type == "linear" :
        return random.randint(1, 10)
    
    if feature_type == "multihot" :
        val1 = random.randint(1, 30)
        val2 = random.randint(1, 30)
        return random.randint(1, 10), min(val1, val2), max(val1, val2)
    
    return

def make_independent_data(datamap, length) :

    data = {}
    
    for k, v in datamap.items() :
        if v == "onehot" :
            data[k] = make_onehot_data(length, random_config(v))
        if v == "linear" :
            
            data[k] = make_linear_data(length, random_config(v))
        if v == "multihot" :
            data[k] = make_multihot(length, random_config(v))
    
    return pd.DataFrame(data)


def make_dataset(target_variable, datamap, length) :
    
    return pd.concat([make_independent_data(datamap, length).assign(y  = i) for i in range(target_variable)])

In [7]:
def search_num_sparse_features(df, datamap, y_col = "y") :
    datamap = datamap.copy()
    datamap.pop("y")
    return max(*[df[k].apply(max).max() if v == "multihot" else df[k].max() for k, v in datamap.items() if v in ["multihot", "onehot"]])

In [8]:
# 모델 인스턴스 생성
num_sparse_features = 6  # Embedding Idx의 최대값 + 1
num_linear_features = 1  # linear feature의 개수 - 개별로 넣으면 1, 묶음이면 len(feature)
embedding_dim = 10
hidden_dim = 20
output_dim = 1  # 예측할 출력의 차원 (예: 회귀의 경우 1, 이진 분류의 경우 1)
print_ok = True

In [10]:
datamap1 = {"a" : "linear", "b" : "multihot", "c" : "onehot", "d" : "multihot", "y" : "onehot"}
datamap2 = {"a" : "linear", "b" : "multihot", "c" : "onehot", "y" : "onehot"}

In [11]:
data1 = make_dataset(3, datamap1, 1000)
data2 = make_dataset(3, datamap2, 1000)

In [16]:
dataset = SiamesDataset(data1, datamap1, data2, datamap2, data1, "y")
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

In [17]:
model1 = EmbeddingModel(datamap1)
model2 = EmbeddingModel(datamap2)

In [18]:
siam_model = SiameseNetwork(model1, model2)
siam_model

SiameseNetwork(
  (model1): EmbeddingModel(
    (a): Linear(in_features=1, out_features=10, bias=True)
    (b): Embedding(6, 10)
    (c): Embedding(6, 10)
    (d): Embedding(6, 10)
    (y): Embedding(6, 10)
    (fc1): Linear(in_features=40, out_features=20, bias=True)
    (fc2): Linear(in_features=20, out_features=1, bias=True)
  )
  (model2): EmbeddingModel(
    (a): Linear(in_features=1, out_features=10, bias=True)
    (b): Embedding(6, 10)
    (c): Embedding(6, 10)
    (y): Embedding(6, 10)
    (fc1): Linear(in_features=30, out_features=20, bias=True)
    (fc2): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [28]:
# 모델 인스턴스 생성
num_sparse_features = max(search_num_sparse_features(data1, datamap1), search_num_sparse_features(data2, datamap2)) + 2  # Embedding Idx의 최대값 + 1
num_linear_features = 1  # linear feature의 개수 - 개별로 넣으면 1, 묶음이면 len(feature)
embedding_dim = 100
hidden_dim = 20
output_dim = 1  # 예측할 출력의 차원 (예: 회귀의 경우 1, 이진 분류의 경우 1)
print_ok = True

In [29]:
num_sparse_features

29

In [30]:
model1 = EmbeddingModel(datamap1)
model2 = EmbeddingModel(datamap2)

In [31]:
siam_model = SiameseNetwork(model1, model2)
siam_model

SiameseNetwork(
  (model1): EmbeddingModel(
    (a): Linear(in_features=1, out_features=100, bias=True)
    (b): Embedding(29, 100)
    (c): Embedding(29, 100)
    (d): Embedding(29, 100)
    (y): Embedding(29, 100)
    (fc1): Linear(in_features=400, out_features=20, bias=True)
    (fc2): Linear(in_features=20, out_features=1, bias=True)
  )
  (model2): EmbeddingModel(
    (a): Linear(in_features=1, out_features=100, bias=True)
    (b): Embedding(29, 100)
    (c): Embedding(29, 100)
    (y): Embedding(29, 100)
    (fc1): Linear(in_features=300, out_features=20, bias=True)
    (fc2): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [35]:
torch.sum(siam_model.model1.b(dataset.__getitem__(0)[0]["b"]), dim = 1)

tensor([ 4.9638, -0.7289,  4.5129], grad_fn=<SumBackward1>)

In [36]:
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(siam_model.parameters(),lr = 0.0005)

counter = []
loss_history = [] 
iteration_number= 0

for epoch in range(0, 100):
    for i, data in enumerate(dataloader):
        x1, x2, y = data
        
        optimizer.zero_grad()
        output1, output2 = siam_model(x1, x2)
        
        loss_contrastive = criterion(output1,output2,y)
        loss_contrastive.backward()
        optimizer.step()
    if epoch %10 == 0 :
        print("Epoch number {}\n Current loss {}\n".format(epoch,loss_contrastive.item()))
        iteration_number +=10
        counter.append(iteration_number)
        loss_history.append(loss_contrastive.item())

Epoch number 0
 Current loss 1985.03466796875

Epoch number 10
 Current loss 73727144.0

Epoch number 20
 Current loss 1222268032.0

Epoch number 30
 Current loss -10822985728.0

Epoch number 40
 Current loss -21899587584.0

Epoch number 50
 Current loss 54278995968.0

Epoch number 60
 Current loss -333290831872.0

Epoch number 70
 Current loss -183532699648.0

Epoch number 80
 Current loss 549518639104.0

Epoch number 90
 Current loss -1022763728896.0



In [37]:
torch.sum(siam_model.model1.b(dataset.__getitem__(0)[0]["b"]), dim = 1)

tensor([ 4.8925, -0.8262,  4.3757], grad_fn=<SumBackward1>)